In [98]:
import pandas as pd 

ranking = pd.read_csv("dataset/fifa_ranking.csv")
champions = pd.read_csv("dataset/champion.csv")
stat = pd.read_csv("dataset/stats.csv")

In [99]:
ranking = ranking[ranking["confederation"] == "CAF"]

In [100]:
ranking = ranking.drop(columns = ["country_abrv","confederation","total_points","previous_points"])
stat = stat.drop(columns=["Rank"])
ranking = ranking.rename(columns = {"country_full" : "Team"})

In [101]:
ranking = ranking.reset_index(drop=True)
ranking.head()

,rank,Team,rank_change,rank_date
0,32.0,Zambia,32,1992-12-31
1,30.0,Algeria,30,1992-12-31
2,27.0,Côte d'Ivoire,27,1992-12-31
3,22.0,Cameroon,22,1992-12-31
4,21.0,Egypt,21,1992-12-31


In [102]:
ranking = ranking.rename(columns = {"rank_date":"year"}) 
ranking = ranking.dropna(axis=0)

In [103]:
ranking["year"] = ranking["year"].astype(str).apply(lambda s: s[:4])
ranking["rank"] = pd.to_numeric(ranking["rank"], errors='coerce').astype('Int64')
ranking["year"] = pd.to_numeric(ranking["year"], errors='coerce').astype('Int64')

In [104]:
rename_dict= {
    'Cabo Verde': 'Cape Verde',
    'Cape Verde Islands': 'Cape Verde',
    'Central African Republic': 'CAR',
    'Congo DR': 'DR Congo',
    "Côte d'Ivoire": 'Ivory Coast',
    'Sao Tome e Principe': 'Sao Tome and Principe',
    'São Tomé e Príncipe': 'Sao Tome and Principe',
    'Swaziland': 'Eswatini',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'The Gambia': 'Gambia',
    'Zaire': 'DR Congo'
}

ranking["Team"] = ranking["Team"].replace(rename_dict)
ranking.head()

,rank,Team,rank_change,year
0,32,Zambia,32,1992
1,30,Algeria,30,1992
2,27,Ivory Coast,27,1992
3,22,Cameroon,22,1992
4,21,Egypt,21,1992


In [105]:
ranking = ranking.merge(stat,how = "left",on = "Team")
ranking.head()

,rank,Team,rank_change,year,Part,Pld,W,D,L,GF,GA,GD,Points
0,32,Zambia,32,1992,18.0,69.0,27.0,19.0,23.0,83.0,70.0,13.0,100.0
1,30,Algeria,30,1992,20.0,80.0,28.0,24.0,28.0,97.0,93.0,4.0,108.0
2,27,Ivory Coast,27,1992,25.0,106.0,48.0,30.0,28.0,152.0,111.0,41.0,174.0
3,22,Cameroon,22,1992,21.0,95.0,46.0,31.0,18.0,142.0,90.0,52.0,169.0
4,21,Egypt,21,1992,26.0,111.0,60.0,24.0,27.0,175.0,97.0,78.0,204.0


In [106]:
rename_dict["Congo (Kinshasa)"] = "Congo"
rename_dict["Congo (Brazzaville)"] = "DR Congo"
champions["winner"] = champions["winner"].replace(rename_dict)
champions["runner-up"] = champions["runner-up"].replace(rename_dict)
champions

,year,winner,runner-up
0,1957,Egypt,Ethiopia
1,1959,Egypt,Sudan
2,1962,Ethiopia,Egypt
3,1963,Ghana,Sudan
4,1965,Ghana,Tunisia
5,1968,Congo,Ghana
6,1970,Sudan,Ghana
7,1972,DR Congo,Mali
8,1974,DR Congo,Zambia
9,1976,Morocco,Guinea


In [107]:
# There are some anomalies with rank_change in top rows to correct
ranking.loc[ranking["year"] == 1992, "rank_change"] = 0
ranking.head()

,rank,Team,rank_change,year,Part,Pld,W,D,L,GF,GA,GD,Points
0,32,Zambia,0,1992,18.0,69.0,27.0,19.0,23.0,83.0,70.0,13.0,100.0
1,30,Algeria,0,1992,20.0,80.0,28.0,24.0,28.0,97.0,93.0,4.0,108.0
2,27,Ivory Coast,0,1992,25.0,106.0,48.0,30.0,28.0,152.0,111.0,41.0,174.0
3,22,Cameroon,0,1992,21.0,95.0,46.0,31.0,18.0,142.0,90.0,52.0,169.0
4,21,Egypt,0,1992,26.0,111.0,60.0,24.0,27.0,175.0,97.0,78.0,204.0


In [108]:
# Adding new columns previous_cup_wins, won_this_year (target output)

def fetchPreviousWins(country,year):
    assert year >= 1957, "Year must be 1957 or later."
    res = champions[(champions["year"] <= year) & (champions["winner"] == country)]
    return len(res)


def WonThisYear(country,year):
    """
    0 means False
    1 means True
    """
    if year == 2024 :
        return 0
    res = champions[(champions["year"] == year) & (champions["winner"] == country)]
    return len(res)

# Some test

print(fetchPreviousWins("Egypt",2024))
print(WonThisYear("Morocco",1976))

7
1


In [109]:
ranking["previous_cup_wins"] = ranking.apply(lambda row: fetchPreviousWins(row["Team"], row["year"]), axis=1)
ranking["won_this_year"] = ranking.apply(lambda row: WonThisYear(row["Team"], row["year"]), axis=1)
ranking.head()

,rank,Team,rank_change,year,Part,Pld,W,D,L,GF,GA,GD,Points,previous_cup_wins,won_this_year
0,32,Zambia,0,1992,18.0,69.0,27.0,19.0,23.0,83.0,70.0,13.0,100.0,0,0
1,30,Algeria,0,1992,20.0,80.0,28.0,24.0,28.0,97.0,93.0,4.0,108.0,1,0
2,27,Ivory Coast,0,1992,25.0,106.0,48.0,30.0,28.0,152.0,111.0,41.0,174.0,1,1
3,22,Cameroon,0,1992,21.0,95.0,46.0,31.0,18.0,142.0,90.0,52.0,169.0,2,0
4,21,Egypt,0,1992,26.0,111.0,60.0,24.0,27.0,175.0,97.0,78.0,204.0,3,0


In [110]:
print(len(set(ranking["Team"])))

54


In [121]:
qualified_teams = [
    "Morocco",
    "Burkina Faso",
    "Cameroon",
    "Algeria",
    "DR Congo",
    "Senegal",
    "Egypt",
    "Angola",
    "Equatorial Guinea",
    "Ivory Coast",
    "Gabon",
    "Uganda",
    "South Africa",
    "Tunisia",
    "Nigeria",
    "Mali",
    "Zambia",
    "Zimbabwe",
    "Comoros",
    "Sudan",
    "Benin",
    "Tanzania",
    "Botswana",
    "Mozambique"
]

# Filtering the data
test = ranking[(ranking["year"] == 2024) & (ranking["Team"].isin(qualified_teams))]

# Dropping the 'won_this_year' column and resetting index
test = test.drop(columns=["won_this_year"])
# Select the last 24 columns
test = test[-24:]

test = test.reset_index(drop=True)

# Filtering data for years <= 2023
data = ranking[ranking["year"] <= 2023]

In [122]:
test

,rank,Team,rank_change,year,Part,Pld,W,D,L,GF,GA,GD,Points,previous_cup_wins
0,67,Burkina Faso,5,2024,13.0,52.0,10.0,17.0,25.0,51.0,78.0,-27.0,47.0,0
1,61,DR Congo,-2,2024,20.0,80.0,21.0,29.0,30.0,94.0,107.0,-13.0,92.0,2
2,59,South Africa,0,2024,11.0,50.0,18.0,16.0,16.0,55.0,48.0,7.0,70.0,1
3,103,Mozambique,-7,2024,5.0,15.0,0.0,4.0,11.0,8.0,33.0,-25.0,4.0,0
4,94,Uganda,2,2024,7.0,23.0,4.0,3.0,16.0,21.0,38.0,-17.0,15.0,0
5,92,Angola,-2,2024,9.0,31.0,7.0,13.0,11.0,39.0,43.0,-4.0,34.0,0
6,91,Benin,-6,2024,4.0,14.0,0.0,5.0,9.0,7.0,24.0,-17.0,5.0,0
7,90,Zambia,4,2024,18.0,69.0,27.0,19.0,23.0,83.0,70.0,13.0,100.0,1
8,89,Equatorial Guinea,10,2024,4.0,19.0,8.0,5.0,6.0,20.0,18.0,2.0,29.0,0
9,83,Gabon,-1,2024,8.0,25.0,7.0,10.0,8.0,24.0,27.0,-3.0,31.0,0


In [123]:
test.to_csv("dataset/test.csv")
data.to_csv("dataset/data.csv")